<a href="https://colab.research.google.com/github/lygitdata/aml_project/blob/main/project2/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [11]:
import csv
import os
import biosppy.signals.ecg as ecg
import biosppy
import neurokit2 as nk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.signal import find_peaks, welch
from scipy.stats import kurtosis, skew
import pywt
import multiprocessing as mp
from tqdm import tqdm
import pickle
from imblearn import over_sampling, pipeline
import lightgbm as lgb
import catboost as cat
from xgboost import XGBRegressor
from sklearn import ensemble, model_selection, preprocessing, svm, linear_model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic
from sklearn.metrics import r2_score, mean_squared_error

# Change this path to the folder that has your data
fpath = "aml_p2/data/"
RANDOM_STATE = 88

# Load data

In [5]:
X_train = np.load(f"{fpath}X_train_selected.npy")
X_test = np.load(f"{fpath}X_test_selected.npy")
y_train = np.load(f"{fpath}y_train.npy")

# Data processing

In [6]:
print(
    "X_train shape: ",
    X_train.shape,
    "\nX_test shape",
    X_test.shape,
)

X_train shape:  (5117, 80) 
X_test shape (3411, 80)


# Model training

In [ ]:
model = pipeline.make_pipeline(
    over_sampling.RandomOverSampler(random_state=RANDOM_STATE),
    preprocessing.StandardScaler(),
    ensemble.StackingClassifier(
        estimators = [
            ("hgb", ensemble.HistGradientBoostingClassifier(l2_regularization=0.2)),
            # ("svr", svm.SVC(C=60.0, kernel='rbf')),
            # ("etr", ensemble.ExtraTreesClassifier()),
            ("lgb", lgb.LGBMClassifier(verbose=0)),
            # ("cat", cat.CatBoostClassifier(verbose=0))
        ],
        final_estimator=linear_model.RidgeClassifierCV()
    )
)

In [33]:
score = model_selection.cross_val_score(
    estimator=model,
    X=X_train,
    y=y_train,
    cv=6,
    n_jobs=-1
)
print("Mean F1 score: ", score.mean(), "\nStd. F1 score: ", score.std())

Mean F1 score:  0.7742823359330137 
Std. F1 score:  0.007105704885220574


# Generate prediction

In [34]:
def create_submission(model, X_train, y_train, X_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred = np.vstack((np.arange(X_test.shape[0]), pred)).T
    np.savetxt("submission.csv", pred, delimiter=",", header="id,y", comments="")


create_submission(model, X_train, y_train, X_test)